# makefile编写

在Unix下的软件编译，你就不能不自己写makefile了，会不会写makefile，从一个侧面说明了一个人是否具备完成大型工程的能力。

makefile关系到了整个工程的编译规则。makefile定义了一系列的规则来指定，哪些文件需要先编译，哪些文件需要后编译，哪些文件需要重新编译，甚至于进行更复杂的功能操作

makefile就像一个Shell脚本一样，其中也可以执行操作系统的命令。

makefile带来的好处就是:“自动化编译”，一旦写好，只需要一个make命令，整个工程完全自动编译，极大的提高了软件开发的效率。

make是一个命令工具，是一个解释makefile中指令的命令工具，

一般大多数IDE都有这个命令，如：Delphi的make，Visual C++的nmake，Linux下GNU的make。可见，makefile都成为了一种在工程方面的编译方法。

### 关于程序的编译和链接

关于程序编译的一些规范和方法，一般来说，无论是C、C++、还是pas，首先要把源文件编译成中间代码文件，在Windows下也就是 .obj 文件，UNIX下是 .o 文件，即 Object File，这个动作叫做编译（compile）。然后再把大量的Object File合成执行文件，这个动作叫作链接（link）。

编译时，编译器需要的是语法的正确，函数与变量的声明的正确。

链接时，主要是链接函数和全局变量，链接器并不管函数所在的源文件，只管函数的中间目标文件（Object File），在大多数时候，由于源文件太多，编译生成的中间目标文件太多，而在链接时需要明显地指出中间目标文件名，这对于编译很不方便，所以，我们要给中间目标文件打个包，在Windows下这种包叫“库文件”（Library File)，也就是 .lib 文件，在UNIX下，是Archive File，也就是 .a 文件。

总结一下，源文件首先会生成中间目标文件，再由中间目标文件生成执行文件。在编译时，编译器只检测程序语法，和函数、变量是否被声明。如果函数未被声明，编译器会给出一个警告，但可以生成Object File。而在链接程序时，链接器会在所有的Object File中找寻函数的实现，如果找不到，那到就会报链接错误码（Linker Error），在VC下，这种错误一般是：Link 2001错误，意思说是说，链接器未能找到函数的实现。你需要指定函数的Object File.

### makefile

make命令执行时，需要一个 Makefile 文件，以告诉make命令需要怎么样的去编译和链接程序。

规则是：
1. 如果这个工程没有编译过，那么我们的所有C文件都要编译并被链接。
2. 如果这个工程的某几个C文件被修改，那么我们只编译被修改的C文件，并链接目标程序。
3. 如果这个工程的头文件被改变了，那么我们需要编译引用了这几个头文件的C文件，并链接目标程序。

Makefile的格式。
```
target ... : prerequisites ...
        command
        ...
        ...
```
* target也就是一个目标文件，可以是Object File，也可以是执行文件。还可以是一个标签（Label）。
* prerequisites就是，要生成那个target所需要的文件或是目标。
* command也就是make需要执行的命令。（任意的Shell命令）

文件依赖关系:
* target这一个或多个的目标文件依赖于prerequisites中的文件，其生成规则定义在command中.
* prerequisites中如果有一个以上的文件比target文件要新的话，command所定义的命令就会被执行

### 一个例子

一个工程有3个头文件，和8个C文件，为了完成前面所述的那三个规则，Makefile应该是下面的这个样子的。

* 3个.h文件: defs.h, command.h, buffer.h
* 8个.c文件: main.c, kbd.c, command.c, display.c, insert.c, search.c, files.c, utils.c

```makefile
edit : main.o kbd.o command.o display.o /
     insert.o search.o files.o utils.o
      cc -o edit main.o kbd.o command.o display.o /
               insert.o search.o files.o utils.o

main.o : main.c defs.h
    cc -c main.c
  
kbd.o : kbd.c defs.h command.h
    cc -c kbd.c
  
command.o : command.c defs.h command.h
    cc -c command.c
  
display.o : display.c defs.h buffer.h
    cc -c display.c
  
insert.o : insert.c defs.h buffer.h
    cc -c insert.c
  
search.o : search.c defs.h buffer.h
    cc -c search.c
  
files.o : files.c defs.h buffer.h command.h
    cc -c files.c
  
utils.o : utils.c defs.h
    cc -c utils.c
  
clean :
    rm edit main.o kbd.o command.o display.o /
       insert.o search.o files.o utils.o
```

反斜杠（/）是换行符的意思

文件名为“Makefile”或“makefile”

该目录下直接输入命令“make”就可以生成执行文件edit

要删除执行文件和所有的中间目标文件，只要执行一下“make clean”就可以了

在这个makefile中，
* 目标文件（target）包含：执行文件edit和中间目标文件（*.o），依赖文件（prerequisites）就是冒号后面的那些 .c 文件和 .h文件。
* 每一个 .o 文件都有一组依赖文件，而这些 .o 文件又是执行文件 edit 的依赖文件。
* 依赖关系的实质上就是说明了目标文件是由哪些文件生成的，换言之，目标文件是哪些文件更新的。

在定义好依赖关系后，后续的那一行定义了如何生成目标文件的操作系统命令，**一定要以一个Tab键作为开头**。

记住，make并不管命令是怎么工作的，他只管执行所定义的命令。
make会比较targets文件和prerequisites文件的修改日期，如果prerequisites文件的日期要比targets文件的日期要新，或者target不存在的话，那么，make就会执行后续定义的命令。

这里要说明一点的是，clean不是一个文件，它只不过是一个动作名字，其冒号后什么也没有，那么，make就不会自动去找文件的依赖性，也就不会自动执行其后所定义的命令。要执行其后的命令，就要在make命令后明显得指出这个lable的名字。这样的方法非常有用，我们可以在一个makefile中定义不用的编译或是和编译无关的命令，比如程序的打包，程序的备份，等等。